<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Access DB2 on Cloud using Python

Estaimted time needed: **15** minutes

## Objectives

After complting this lab you will be able to:

-   Create a table
-   Insert data into the table
-   Query data from the table
-   Retrieve the result set into a pandas dataframe
-   Close the database connection


##  Task 1: Import the ibm_db Python library

<code> ibm_db </code> API provides a variety of useful functions for accessing and manipulating data in an IBM data server databse, including functions for connecting to a database, preparing and issuing SQL statements, fetching rows from result sets, calling stored procedures, committing and rolling back transactions, handling errors, and retrieving metadata.

In [30]:
# Import ibm_db 

import ibm_db

## Task 2: Identify the database connection credentials

Connection requires following informations: DRIVER, DATABASE NAME, HOST DNS NAME OR IP ADDRESS, HOST PORT, CONNECTION PROTOCOL, USER ID, USER PASSWORD 

Load the above mentioned credentials from the first lab in this course.

In [31]:
# Code baby code

dsn_hostname = "dashdb-txn-sbox-yp-lon02-04.services.eu-gb.bluemix.net"
dsn_database = "BLUDB"
dsn_uid = "xhc04162"
dsn_port = "50000"
dsn_protocol = "TCPIP"
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_pwd = "jkwn881vt3-9hw52"


## Task 3: Create the database connection

Ibm_db API uses the IBM Data Server Driver for ODBC and CLI APIs to connect to IBM DB2 and Informix.

Create the data base connection

In [32]:
# Database connection

dsn = (
    "DRIVER = {0};"
    "DATABASE = {1};"
    "HOSTNAME = {2};"
    "PORT = {3};"
    "PROTOCOL = {4};"
    "UID = {5};"
    "PWD = {6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)
try:
    conn = ibm_db.connect(dsn, "", "")
    print("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)
except:
    print("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  xhc04162 on host:  dashdb-txn-sbox-yp-lon02-04.services.eu-gb.bluemix.net


## Task 4: Create a table in the database

We create a table in the database:

In [33]:
# Drop the table in case it has been already made

dropQuery = "drop table INSTRUCTOR"

# Execute the drop statement
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

### Construct a table

In [34]:
# Construct the create table ddl statement -- replace the .. with rest of the statement

createQuery = "create table INSTRUCTOR(id INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))"

# Now fill in the name of the method and execute the statement

createStmt = ibm_db.exec_immediate(conn, createQuery)

## Task 5: Insert data into the table

We will insert some rows of data into the table, for instructor Rav Ahuja

In [36]:
#Construct the query - replace ... with the insert statement

insertQuery = "insert into INSTRUCTOR values (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"

#Execeute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

Exception: [IBM][CLI Driver][DB2/LINUXX8664] SQL0803N  One or more values in the INSERT statement, UPDATE statement, or foreign key update caused by a DELETE statement are not valid because the primary key, unique constraint or unique index identified by "1" constrains table "XHC04162.INSTRUCTOR" from having duplicate values for the index key.  SQLSTATE=23505 SQLCODE=-803

Use single query to insert the remaining two rows of data

In [40]:
insertQuery2 = "insert into INSTRUCTOR values (2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"

#time to execute

insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)

## Task 6: Query data in the table

In this step we will retrieve data we inserted into the INSTRUCTOR table.

In [55]:
# Construct the query that retrieves all rows form the INSTRUCTOR table

selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

In [56]:

#Fetch the Dictionary (for the first row only)

ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

#### Fetch the rest of the rows and print the ID and FNAME for those rows



In [57]:
while ibm_db.fetch_row(selectStmt) != False:
    print(" ID:", ibm_db.result(selectStmt, 0), " FNAME:", ibm_db.result(selectStmt, "FNAME"))

 ID: 2  FNAME: Raul
 ID: 3  FNAME: Hima


Bonus: now write and execute an update statement that changes the Rav's CITY to MOOSETOWN

In [ ]:
updateQuery = "update INSTRUCTOR set CITY ='MOOSETOWN' where FNAME = 'Rav'"

# Task 7: Retrieve data into Pandas

In this step we will retrieve the contents of the INSTRUCTOR table into a Pandas dataframe

In [58]:
import pandas
import ibm_db_dbi

In [59]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [64]:
#query statement to retrieve all rows in INSTRUCTOR table

selectQuery = "select * from INSTRUCTOR"

#RETRIEVE THE QUERY RESULTS INTO A PANDAS DATAFRAME
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for third row in the pandas data frame 
pdf.LNAME[2]





'Vasudevan'

In [65]:
#print the entire dataframe
pdf


,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,TORONTO,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


## Task 8: Close the Connection

In [66]:
pdf.shape

(3, 5)

checking the normal pandas fuction

Time to close

In [67]:
ibm_db.close(conn)

True